## HW 2. Bank digital campaign analysis

--------

__Authors:__ Turakulov A., Sitdikova Y., Rakhimkulova A., Yakovlev P.    
__Group:__ BASB212  

--------

### Task description

You are provided with data describing the results of an email and push campaigns of a bank.

__The offer was the following:__  
The client should open a new Visa card and make 3 transactions at 500 rub at least each during a 14-day
period. If he does so, he receives a 5% cashback for all transactions made using this card during the next
month.

__The datasets contain the following information:__  
1. The funnel metrics of a campaign (“visa_communications”)
2. The statistic of visa card openings (“card_openings”, “card_openings_not_particip”)
3. The transactional data within the campaign period (“transactions”, “transactions_non_particip”)
4. The features for generating the synthetic control group (“features”).

__The goal of the homework:__  
To analyze a dataset containing customers' responses on a campaign, conclude whether the campaign
was successful or not by calculating the financial effect using Python. The bank gets 1% of any
transaction amount as income. 

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 18 kB/s 
     |████████████████████████████████| 199 kB 60.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=34464ffdd65b5b7a3232a13e38a775851075368c3ae9b6c041ce5a4376d01ee0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession , SQLContext
from pyspark.sql.functions import sum 

spark = SparkSession.builder.appName("HW2").getOrCreate()
sqlContext = SQLContext(spark)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_visa_comms = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hw2_marketing/visa_communications.csv')
df_visa_comms.head()

,channel_type,contact_dt,control_group_flg,sent,delivered,opened,clicked,blocked,cust_id
0,Popup,2021-08-23,0,1,1,1,0,0,5d0a0cae173c27947838b9f3ad1282fa30771272747ede...
1,EMAIL,2021-08-12,0,1,1,0,0,0,b70d7c45f4d1724acdba89bead5941f13d93406bcbb1d7...
2,Popup,2021-08-11,0,1,1,1,0,0,1b3705d419798b8723884b643dc535ae7fc7623d6f4f7c...
3,PUSH,2021-08-12,0,0,0,0,0,0,0064bbbb2fd516698fd113adcabef8bb52824536512e8e...
4,PUSH,2021-08-12,0,0,0,0,0,0,3f0be853435987e9aff8cba237e8c19334708a996cf696...


In [5]:
df_crd_opns = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hw2_marketing/card_openings.csv')
df_crd_opns.head()

,cust_id,open_dt,contact_dt
0,2537ec14c2e5b8607ea67dfe09ec687205c59ed152407f...,2021-08-19,2021-08-12
1,b6bc9799397f07c8304673d4d51e7788a143ecb710452e...,2021-08-16,2021-08-13
2,d09ee17e668f229697a15a940915714b838826305e6973...,2021-08-21,2021-08-13
3,2323b4ae10e99e72160a74c5b42c38e47f2bc0692ef0a9...,2021-08-23,2021-08-13
4,b056b9822c2208a83eeb7150ee4be30d7088b07fed0ec5...,2021-08-23,2021-08-13


In [6]:
df_crd_opns_no = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hw2_marketing/card_openings_non_participants.csv')
df_crd_opns_no.head()

,cust_id,open_dt,contact_dt
0,00142a5cfe4dd6ac883e4dc0ba888d6266ce99257e0825...,2021-08-26,0
1,00b037b74b45a2a0c9dc64b59a2587dd79dabc93e1d84b...,2021-08-12,0
2,00d312b7591fdc7d5607feac6ebcff5223d2162bff5248...,2021-08-26,0
3,01965e567e31d90ae31b5476a7bdb44b76ebf0d3950cf4...,2021-08-04,0
4,02a874b8110d3a04c551a88d8e35529e2faf553d8512e5...,2021-08-04,0


In [7]:
df_trans = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hw2_marketing/visa_transactions.csv')
df_trans.head()

,cust_id,transaction_dt,mcc_id,txn_sum
0,db5e62076fad2729571f211d7ca4d5be1ba71e7563efbb...,2021-09-09,5399,2322.00
1,9fc01d326a0adcb9caaabe298064b8a26aa711878cf89f...,2021-09-07,5533,1245.00
2,7afcb430aaf9baa1786d36ffcdf3efec0f4f6fe41b32fc...,2021-08-30,5921,29.99
3,79a2d16cd93c5d15120f30f4a6a0b39060f6cdde7486c8...,2021-09-04,5211,110.00
4,0c10b18ba26cb55e4b1aed9146cf359d034fa872feeba6...,2021-08-31,5441,45.00


In [8]:
df_trans_no = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hw2_marketing/visa_transactions_non_particip.csv')
df_trans_no.head()

,cust_id,transaction_dt,mcc_id,txn_sum
0,7ab840dd10c9eec7ba477160c74eaff315746f5db370c9...,2021-08-28,5499,645.00
1,b920f5f439d996cb87ff138610bd123bcd03dae7bc4602...,2021-09-15,5921,304.56
2,e14bc4726c13b92a503e2089654a4abf05f1ce9effee52...,2021-09-09,5912,1249.40
3,a6d6a39ca52e7e1b6743274c94c1ed3b1ec06d7a726a51...,2021-08-30,5411,346.27
4,380a3ce00a9dff9daccb414c20dcc82c30100c23f23265...,2021-08-21,5812,35.00


In [9]:
df_ftrs = spark.read.option("header", True).csv('/content/drive/MyDrive/Colab Notebooks/hw2_marketing/features.csv')
df_ftrs.show()

+--------------------+----------+--------------------+----------+--------------+------+------+
|             cust_id|  mnth_gen|            turnover|involve_cd|client_segment|gender|age_gr|
+--------------------+----------+--------------------+----------+--------------+------+------+
|ad767176b86d500e5...|2021-06-01|17764.83000000000...|         1|          MASS|     F| 25-40|
|cc7c069e1866b0e14...|2021-06-01|18817.45000000000...|         1|          MASS|     M| 25-40|
|914c9e80623d8fdec...|2021-01-01|19198.74000000000...|         1|     MVS LIGHT|     F| 25-40|
|e45e78863f239d393...|2021-04-01|114637.5400000000...|         1|     MVS UPPER|     F| 40-60|
|2201a68b8c83f098a...|2021-07-01|112603.0100000000...|         0|     MVS LIGHT|     F| 25-40|
|7b7829394d7897140...|2021-01-01|17576.22000000000...|         1|     MVS LIGHT|     F| 40-60|
|ecad93826320a5980...|2021-07-01|120845.4500000000...|         0|     MVS UPPER|     M| 40-60|
|96c63552c78c342a4...|2021-03-01|5878.500000000000

### 1. Make the data preprocessing (1 point)

In [10]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Misses', 1 : '% from all data'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% from all data', ascending=False).round(1)
    print ("Dataframe contains " + str(df.shape[1]) + " columns.\n"      
        "Overall " + str(mis_val_table_ren_columns.shape[0]) +
          " columns with missing values.")
    return mis_val_table_ren_columns

In [11]:
missing_values_table(df_visa_comms).style.background_gradient(cmap='Reds')

Dataframe contains 9 columns.
Overall 0 columns with missing values.


,Misses,% from all data


In [12]:
missing_values_table(df_crd_opns).style.background_gradient(cmap='Reds')

Dataframe contains 3 columns.
Overall 0 columns with missing values.


,Misses,% from all data


In [13]:
missing_values_table(df_crd_opns_no).style.background_gradient(cmap='Reds')

Dataframe contains 3 columns.
Overall 0 columns with missing values.


,Misses,% from all data


In [14]:
missing_values_table(df_trans).style.background_gradient(cmap='Reds')

Dataframe contains 4 columns.
Overall 0 columns with missing values.


,Misses,% from all data


In [15]:
missing_values_table(df_trans_no).style.background_gradient(cmap='Reds')

Dataframe contains 4 columns.
Overall 0 columns with missing values.


,Misses,% from all data


In [16]:
missing_values_table(df_ftrs.limit(100000).toPandas()).style.background_gradient(cmap='Reds')

Dataframe contains 7 columns.
Overall 0 columns with missing values.


,Misses,% from all data


In [17]:
print(f'Shape of DF (df_ftrs) : {df_ftrs.count()}')
print(df_ftrs.limit(1000).toPandas().info())
df_ftrs.limit(1000).toPandas().describe()

Shape of DF (df_ftrs) : 31418100
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cust_id         1000 non-null   object
 1   mnth_gen        1000 non-null   object
 2   turnover        1000 non-null   object
 3   involve_cd      1000 non-null   object
 4   client_segment  1000 non-null   object
 5   gender          1000 non-null   object
 6   age_gr          1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB
None


,cust_id,mnth_gen,turnover,involve_cd,client_segment,gender,age_gr
count,1000,1000,1000,1000,1000,1000,1000
unique,1000,10,991,2,4,3,5
top,ad767176b86d500e5b8421315731388ff6c5ec93079ec9...,2021-09-01,0,1,MASS,F,25-40
freq,1,117,10,569,437,565,494


Converting date frames to pyspark

In [18]:
print(f'Shape of DF (df_visa_comms) : {df_visa_comms.shape}')
print(df_visa_comms.info())
print(df_visa_comms.describe())
df_visa_comms['contact_dt'] = pd.to_datetime(df_visa_comms['contact_dt'], errors='coerce')
print(df_visa_comms.head())
df_visa_comms = spark.createDataFrame(df_visa_comms) 
df_visa_comms.printSchema()

Shape of DF (df_visa_comms) : (1456826, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456826 entries, 0 to 1456825
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   channel_type       1456826 non-null  object
 1   contact_dt         1456826 non-null  object
 2   control_group_flg  1456826 non-null  int64 
 3   sent               1456826 non-null  int64 
 4   delivered          1456826 non-null  int64 
 5   opened             1456826 non-null  int64 
 6   clicked            1456826 non-null  int64 
 7   blocked            1456826 non-null  int64 
 8   cust_id            1456826 non-null  object
dtypes: int64(6), object(3)
memory usage: 100.0+ MB
None
       control_group_flg          sent     delivered        opened  \
count       1.456826e+06  1.456826e+06  1.456826e+06  1.456826e+06   
mean        7.048199e-02  5.635889e-01  5.413014e-01  2.176272e-01   
std         2.559577e-01  4.959401e-01 

In [19]:
print(f'Shape of DF (df_crd_opns) : {df_crd_opns.shape}')
print(df_crd_opns.info())
print(df_crd_opns.describe())
df_crd_opns['open_dt'] = pd.to_datetime(df_crd_opns['open_dt'], errors='coerce')
df_crd_opns['contact_dt'] = pd.to_datetime(df_crd_opns['contact_dt'], errors='coerce')
print(df_crd_opns.head())
df_crd_opns = spark.createDataFrame(df_crd_opns) 
df_crd_opns.printSchema()

Shape of DF (df_crd_opns) : (22995, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22995 entries, 0 to 22994
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cust_id     22995 non-null  object
 1   open_dt     22995 non-null  object
 2   contact_dt  22995 non-null  object
dtypes: object(3)
memory usage: 539.1+ KB
None
                                                  cust_id     open_dt  \
count                                               22995       22995   
unique                                              21424          21   
top     fa4aeb57cc6f9cf63661203bfa17635e247cbca3d2c018...  2021-08-12   
freq                                                   12        2102   

        contact_dt  
count        22995  
unique          15  
top     2021-08-12  
freq         18466  
                                             cust_id    open_dt contact_dt
0  2537ec14c2e5b8607ea67dfe09ec687205c59ed152407f... 2021-08

In [20]:
print(f'Shape of DF (df_trans) : {df_trans.shape}')
print(df_trans.info())
print(df_trans.describe())
df_trans['transaction_dt'] = pd.to_datetime(df_trans['transaction_dt'], errors='coerce')
print(df_trans.head())
df_trans = spark.createDataFrame(df_trans) 
df_trans.printSchema()

Shape of DF (df_trans) : (396564, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396564 entries, 0 to 396563
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   cust_id         396564 non-null  object 
 1   transaction_dt  396564 non-null  object 
 2   mcc_id          396564 non-null  int64  
 3   txn_sum         396564 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 12.1+ MB
None
              mcc_id       txn_sum
count  396564.000000  3.965640e+05
mean     5525.351963  8.075815e+02
std       751.054988  1.451721e+04
min       742.000000  4.000000e-02
25%      5411.000000  1.140000e+02
50%      5462.000000  2.678550e+02
75%      5814.000000  6.285700e+02
max      9998.000000  8.490000e+06
                                             cust_id transaction_dt  mcc_id  \
0  db5e62076fad2729571f211d7ca4d5be1ba71e7563efbb...     2021-09-09    5399   
1  9fc01d326a0adcb9caaabe298064b8a26

There were duplicate clients in the notifications based on control_group_flg

In [21]:
df_visa_comms.filter(df_visa_comms.cust_id =='804e1c3c26f33c641304b75298f8f019135d54f4e21f0bd87b2d4d474c6d5c7f').show(100, False)

+------------+-------------------+-----------------+----+---------+------+-------+-------+----------------------------------------------------------------+
|channel_type|contact_dt         |control_group_flg|sent|delivered|opened|clicked|blocked|cust_id                                                         |
+------------+-------------------+-----------------+----+---------+------+-------+-------+----------------------------------------------------------------+
|EMAIL       |2021-08-13 00:00:00|0                |1   |1        |0     |0      |0      |804e1c3c26f33c641304b75298f8f019135d54f4e21f0bd87b2d4d474c6d5c7f|
|EMAIL       |2021-08-12 00:00:00|1                |0   |0        |0     |0      |0      |804e1c3c26f33c641304b75298f8f019135d54f4e21f0bd87b2d4d474c6d5c7f|
|PUSH        |2021-08-12 00:00:00|0                |0   |0        |0     |0      |0      |804e1c3c26f33c641304b75298f8f019135d54f4e21f0bd87b2d4d474c6d5c7f|
+------------+-------------------+-----------------+----+-------

In [22]:
df_visa_comms.registerTempTable("comms")

comms = sqlContext.sql("""select count(*)
                        from (SELECT cust_id , count(distinct control_group_flg) as cnt
                            FROM comms
                            group by cust_id
                            having count(distinct control_group_flg) > 1
                            ) a
                        """)
comms.show()

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+--------+
|count(1)|
+--------+
|   21950|
+--------+



Removing duplicate clients from all dataframes

In [23]:
df_crd_opns.registerTempTable("crds")

df_crd_opns = sqlContext.sql("""SELECT * FROM crds WHERE cust_id not in 
                                (select distinct cust_id
                                from (SELECT cust_id , count(distinct control_group_flg) as cnt
                                    FROM comms
                                    group by cust_id
                                    having count(distinct control_group_flg) > 1
                                    ) a
                                )
                            """)
df_crd_opns.show()

+--------------------+-------------------+-------------------+
|             cust_id|            open_dt|         contact_dt|
+--------------------+-------------------+-------------------+
|2537ec14c2e5b8607...|2021-08-19 00:00:00|2021-08-12 00:00:00|
|b6bc9799397f07c83...|2021-08-16 00:00:00|2021-08-13 00:00:00|
|d09ee17e668f22969...|2021-08-21 00:00:00|2021-08-13 00:00:00|
|2323b4ae10e99e721...|2021-08-23 00:00:00|2021-08-13 00:00:00|
|7a5665b615fce7d6d...|2021-08-18 00:00:00|2021-08-12 00:00:00|
|08f53c97adf38bda6...|2021-08-12 00:00:00|2021-08-12 00:00:00|
|8394c7e7466057572...|2021-08-23 00:00:00|2021-08-12 00:00:00|
|7c2616d054c0a9940...|2021-08-19 00:00:00|2021-08-12 00:00:00|
|bad095ff8563cb0ef...|2021-08-13 00:00:00|2021-08-12 00:00:00|
|7e9760e1298b77617...|2021-08-17 00:00:00|2021-08-12 00:00:00|
|df424d735cd7cfac3...|2021-08-16 00:00:00|2021-08-12 00:00:00|
|fcc4dd1fd016837f6...|2021-08-25 00:00:00|2021-08-12 00:00:00|
|a84d871095e1c2ac7...|2021-08-12 00:00:00|2021-08-12 00

In [24]:
df_trans.registerTempTable("trans")

df_trans = sqlContext.sql("""SELECT * FROM trans WHERE cust_id not in 
                                (select distinct cust_id
                                from (SELECT cust_id , count(distinct control_group_flg) as cnt
                                    FROM comms
                                    group by cust_id
                                    having count(distinct control_group_flg) > 1
                                    ) a
                                )
                            """)
df_trans.show()

+--------------------+-------------------+------+-------+
|             cust_id|     transaction_dt|mcc_id|txn_sum|
+--------------------+-------------------+------+-------+
|db5e62076fad27295...|2021-09-09 00:00:00|  5399| 2322.0|
|9fc01d326a0adcb9c...|2021-09-07 00:00:00|  5533| 1245.0|
|7afcb430aaf9baa17...|2021-08-30 00:00:00|  5921|  29.99|
|79a2d16cd93c5d151...|2021-09-04 00:00:00|  5211|  110.0|
|0c10b18ba26cb55e4...|2021-08-31 00:00:00|  5441|   45.0|
|80dee4219eef1bbf3...|2021-09-15 00:00:00|  5411| 263.65|
|188c790564fd31843...|2021-09-03 00:00:00|  5411|  129.0|
|1ef28607bf5bd0ffe...|2021-09-08 00:00:00|  5411| 1411.0|
|ec1acb6ccaaf88034...|2021-08-22 00:00:00|  5977|  687.0|
|d33ced03426b6230f...|2021-08-28 00:00:00|  4722|36360.0|
|5ecf29bbfe9925258...|2021-09-11 00:00:00|  5691|  278.0|
|42dfc5265736e9bd3...|2021-08-30 00:00:00|  5499| 219.06|
|773d82f35b0f87328...|2021-08-28 00:00:00|  5912|   55.0|
|5cea8338ac5857a0a...|2021-08-29 00:00:00|  5541|  999.8|
|ba7afae0a7dfa

In [25]:
df_ftrs.registerTempTable("ftrs")

df_ftrs = sqlContext.sql("""SELECT * FROM ftrs WHERE cust_id not in 
                                (select distinct cust_id
                                from (SELECT cust_id , count(distinct control_group_flg) as cnt
                                    FROM comms
                                    group by cust_id
                                    having count(distinct control_group_flg) > 1
                                    ) a
                                )
                            """)
df_ftrs.show()

+--------------------+----------+--------------------+----------+--------------+------+------+
|             cust_id|  mnth_gen|            turnover|involve_cd|client_segment|gender|age_gr|
+--------------------+----------+--------------------+----------+--------------+------+------+
|ad767176b86d500e5...|2021-06-01|17764.83000000000...|         1|          MASS|     F| 25-40|
|cc7c069e1866b0e14...|2021-06-01|18817.45000000000...|         1|          MASS|     M| 25-40|
|914c9e80623d8fdec...|2021-01-01|19198.74000000000...|         1|     MVS LIGHT|     F| 25-40|
|e45e78863f239d393...|2021-04-01|114637.5400000000...|         1|     MVS UPPER|     F| 40-60|
|7b7829394d7897140...|2021-01-01|17576.22000000000...|         1|     MVS LIGHT|     F| 40-60|
|ecad93826320a5980...|2021-07-01|120845.4500000000...|         0|     MVS UPPER|     M| 40-60|
|96c63552c78c342a4...|2021-03-01|5878.500000000000...|         0|          MASS|     F| 25-40|
|6b13c0fed87df6302...|2021-01-01|16811.72000000000

In [26]:
df_visa_comms = sqlContext.sql("""SELECT * FROM comms WHERE cust_id not in 
                                (select distinct cust_id
                                from (SELECT cust_id , count(distinct control_group_flg) as cnt
                                    FROM comms
                                    group by cust_id
                                    having count(distinct control_group_flg) > 1
                                    ) a
                                )
                            """)
df_visa_comms.show()

+------------+-------------------+-----------------+----+---------+------+-------+-------+--------------------+
|channel_type|         contact_dt|control_group_flg|sent|delivered|opened|clicked|blocked|             cust_id|
+------------+-------------------+-----------------+----+---------+------+-------+-------+--------------------+
|       Popup|2021-08-23 00:00:00|                0|   1|        1|     1|      0|      0|5d0a0cae173c27947...|
|       EMAIL|2021-08-12 00:00:00|                0|   1|        1|     0|      0|      0|b70d7c45f4d1724ac...|
|       Popup|2021-08-11 00:00:00|                0|   1|        1|     1|      0|      0|1b3705d419798b872...|
|        PUSH|2021-08-12 00:00:00|                0|   0|        0|     0|      0|      0|0064bbbb2fd516698...|
|        PUSH|2021-08-12 00:00:00|                0|   0|        0|     0|      0|      0|3f0be853435987e9a...|
|       Popup|2021-08-24 00:00:00|                0|   1|        1|     1|      0|      0|c645c4639726e6

Let's check it again

In [27]:
df_visa_comms.registerTempTable("comms")
df_crd_opns.registerTempTable("crds")
df_trans.registerTempTable("trans")
df_ftrs.registerTempTable("ftrs")

comms = sqlContext.sql("""select count(*)
                        from (SELECT cust_id , count(distinct control_group_flg) as cnt
                            FROM comms
                            group by cust_id
                            having count(distinct control_group_flg) > 1
                            ) a
                        """)
comms.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



Let's create a flat sample of clients with a flag for fulfilling campaign conditions

In [28]:
results = sqlContext.sql("""WITH single_cards as (
                                SELECT crds.cust_id , cm.control_group_flg, min(crds.open_dt) as open_dt
                                FROM crds 
                                INNER JOIN (SELECT cust_id , control_group_flg, max(contact_dt) as max_contact_dt FROM comms GROUP BY cust_id , control_group_flg) as cm 
                                ON crds.cust_id = cm.cust_id AND cm.control_group_flg = 0  and max_contact_dt <= crds.open_dt
                                GROUP BY crds.cust_id, cm.control_group_flg
                            )
                            SELECT a.cust_id 
                                , a.open_dt 
                                , sum(coalesce(all_t.txn_sum,0)) as all_txn_sum
                                , a.campaign_txn_sum
                                , a.all_trans_count as campaign_trans_count
                                , a.flag_success_campaign
                                , a.control_group_flg
                            FROM (
                                SELECT cust_id 
                                    , open_dt 
                                    , sum(CASE WHEN main.all_trans_count >= 3 
                                                THEN txn_sum
                                                ELSE 0
                                          END) AS campaign_txn_sum
                                    , all_trans_count
                                    , CASE WHEN main.all_trans_count >= 3 
                                            THEN 1
                                            ELSE 0
                                      END AS flag_success_campaign
                                    , control_group_flg
                                FROM (
                                    SELECT crds.cust_id 
                                        , crds.open_dt 
                                        , trans.transaction_dt 
                                        , trans.mcc_id 
                                        , trans.txn_sum
                                        , count(trans.cust_id) OVER (PARTITION BY crds.cust_id) as all_trans_count
                                        , crds.control_group_flg
                                    FROM single_cards as crds
                                    LEFT JOIN trans 
                                    ON crds.cust_id = trans.cust_id  
                                    AND trans.txn_sum >= 500 
                                    AND crds.open_dt <= trans.transaction_dt
                                    AND trans.transaction_dt < date_add(crds.open_dt, 14) 
                                ) main
                                GROUP BY cust_id 
                                    , open_dt 
                                    , all_trans_count
                                    , CASE WHEN main.all_trans_count >= 3 
                                            THEN 1
                                            ELSE 0
                                      END
                                    , control_group_flg
                            ) a
                            LEFT JOIN trans as all_t
                                ON a.cust_id = all_t.cust_id 
                            group by a.cust_id 
                                , a.open_dt 
                                , a.campaign_txn_sum
                                , a.all_trans_count
                                , a.flag_success_campaign
                                , a.control_group_flg
                            """)
results.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[cust_id#119, open_dt#429, campaign_txn_sum#424, all_trans_count#423L, flag_success_campaign#425, control_group_flg#432L], functions=[sum(coalesce(txn_sum#451, 0.0))])
   +- HashAggregate(keys=[cust_id#119, open_dt#429, knownfloatingpointnormalized(normalizenanandzero(campaign_txn_sum#424)) AS campaign_txn_sum#424, all_trans_count#423L, flag_success_campaign#425, control_group_flg#432L], functions=[partial_sum(coalesce(txn_sum#451, 0.0))])
      +- Project [cust_id#119, open_dt#429, campaign_txn_sum#424, all_trans_count#423L, flag_success_campaign#425, control_group_flg#432L, txn_sum#451]
         +- SortMergeJoin [cust_id#119], [cust_id#448], LeftOuter
            :- Sort [cust_id#119 ASC NULLS FIRST], false, 0
            :  +- HashAggregate(keys=[cust_id#119, open_dt#429, all_trans_count#423L, _groupingexpression#473, control_group_flg#432L], functions=[sum(CASE WHEN (all_trans_count#423L >= 3) THEN txn_su

The dataframe of the results includes the fields: 
- cust_id 
- open_dt
- all_txt_sum (Sum of all transactions) 
- campaign_txt_sum (The amount of transactions during the execution of the campaign)
- campaign_trans_count (The number of transactions during the execution of the campaign)
- flag_success_campaign (Campaign execution flag. 1 - the conditions are met. 0 - conditions are not met)

Campaign execution condition:
The client must open a new Visa card and make 3 transactions in the amount of at least 500 rubles each within a 14-day period.

In [29]:
results.sort("cust_id").limit(1000).show(100, False)

+----------------------------------------------------------------+-------------------+------------------+------------------+--------------------+---------------------+-----------------+
|cust_id                                                         |open_dt            |all_txn_sum       |campaign_txn_sum  |campaign_trans_count|flag_success_campaign|control_group_flg|
+----------------------------------------------------------------+-------------------+------------------+------------------+--------------------+---------------------+-----------------+
|0000d77b0a84faaed1feb633819762454e11b716c7a9e4090f8c27ebc797f7f2|2021-08-20 00:00:00|6081.48           |0.0               |0                   |0                    |0                |
|0001f693f461c4a742c3bec39528bab626212dab10d27e2b638bdd5f19bd1064|2021-08-17 00:00:00|5748.05           |0.0               |0                   |0                    |0                |
|000238bd9e5c9d21a742471123ae9dcb4d391273223890c465945375b69d3aa2|2021

# Key indicators
For analysis we will analyze rates of email funnel indicators: Delivery rate, open rate, click rate, etc. And also we will consider conversion and revenue.

# Email funnel calculations

__Delivery rate__ ( DR = number of delivered emails ÷ number of sent emails × 100)

In [30]:
df_visa_comms.filter(df_visa_comms.sent == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.sent).count()

504706

In [31]:
df_visa_comms.filter(df_visa_comms.delivered == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.delivered).count()

504706

In [32]:
DR = ((df_visa_comms.filter(df_visa_comms.delivered == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.delivered).count())/(df_visa_comms.filter(df_visa_comms.sent == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.sent).count()))*100

In [33]:
np.round(DR,1)

100.0

**Open rate** OR = open_emails/delivered * 100%


In [34]:
df_visa_comms.filter(df_visa_comms.opened == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.opened).count()

51816

In [35]:
OR = ((df_visa_comms.filter(df_visa_comms.opened == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.opened).count())/(df_visa_comms.filter(df_visa_comms.delivered == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.delivered).count()))*100

In [36]:
np.round(OR,1)

10.3

**Click Rate** = clicked_emails/delivered * 100%

In [37]:
df_visa_comms.filter(df_visa_comms.clicked == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.clicked).count()

3053

In [38]:
CR = ((df_visa_comms.filter(df_visa_comms.clicked == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.clicked).count())/(df_visa_comms.filter(df_visa_comms.delivered == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.delivered).count()))*100

In [39]:
np.round(CR,1)

0.6

**ConvR** = made target action / delivered emails * 100%

In [40]:
results.filter(results.flag_success_campaign == 1).select(results.flag_success_campaign).count()

5369

In [41]:
ConvR =((results.filter(results.flag_success_campaign == 1).select(results.flag_success_campaign).count())/(df_visa_comms.filter(df_visa_comms.delivered == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.delivered).count())) * 100

In [42]:
np.round(ConvR,1)

1.1

**Revenue_per_email** sum of revenue from campaign/delivered emails * 100

In [43]:

(results.select(results.campaign_txn_sum).agg(sum("campaign_txn_sum")).show()) 

+---------------------+
|sum(campaign_txn_sum)|
+---------------------+
|  9.664379336999992E7|
+---------------------+



In [44]:
np.round(9.664379336999992E7 / (df_visa_comms.filter(df_visa_comms.delivered == 1).filter(df_visa_comms.channel_type == 'EMAIL').select(df_visa_comms.delivered).count()),1)

191.5

Based on the results of analytics, we have identified DR. We see in absolute values that the number of emails delivered is equal to the number of emails sent and DR=100%. This is very good, because we see that the letters were not lost and reached the addressees. It is important to note that a high speed of delivery does not mean a high place in the Inbox – your email can be delivered, but end up in junk mail folders where the recipient will not notice it. Therefore, it is important to calculate other indicators.

In this case, the bounce rate is also considered.
The bounce rate errors can be of two types:

Soft (Soft bounces). The error occurred on the mail server or on your side. Such errors can be corrected and still make sure that the letter reaches the client.

Hard (Strong bounces). It is impossible to deliver the letter, for example, you have the wrong address or the mailbox has been deleted. If this indicator is close to 4-5%, you need to solve technical problems before continuing work.

The open rate is quite low when compared with benchmarks from HubSpot and campaign monitoring **[ 1, 2 ]** for the banking industry. There they are kept in the range from 18 to 27 percent. 

The click rate seems more normal based on the benchmark of 1.27% **[3]**, but still lower.

The CR seems low relative to benchmarks, but it needs to be looked at in dynamics also at the company **[4]**.

It is important to note that RPE calculates the profit received/ (the number of emails sent is the number of undelivered emails). However, since we have we have the flag delivered and the quantity with the flag this matches the sent formula we have reduced. In fact, according to statistics, this indicator is tracked less often than others. RPE can be crucial in determining the success of a company **[5]**. It helps to evaluate the effectiveness of mailing, but without taking into account data on the cost of email. In this sense, RPE is an alternative to more accurate metrics, such as return on investment. It is easier and faster to calculate RPE - to calculate the return on investment, we need all the costs for the period. Plus, it is not always possible — for example, if there are several businesses or business units and there is no clear structuring of income and expenses. 

RPE is a non—independent metric. To interpret it more accurately, we need to analyze it together with the total revenue from the mailing list. Without this, it is impossible to say unequivocally that the growth of the RPE is always good, and the fall is always bad. Depending on how both indicators change, we can assume the reason and draw conclusions. If we had had an RPE for the last newsletter and we understood its boundaries, we could compare them in dynamics.

Even if the figures are lower than expected, this does not always mean that the mailing list is bad. Indicators depends on many factors: for example, on the information field in which readers are located, or on how the database was compiled. Perhaps it was a cold base of clients in our marketing campaign, in general, the indicators were much lower.

Therefore, there is a general recommendation: first of all, focus on how the indicators change.

Sources:
1. https://www.campaignmonitor.com/resources/guides/email-marketing-benchmarks/
2. https://blog.hubspot.com/sales/average-email-open-rate-benchmark?_ga=2.44782887.247958909.1559198466-1080458066.1558941446
3. https://www.smartinsights.com/email-marketing/email-communications-strategy/statistics-sources-for-email-marketing/
4. https://exponea.com/blog/email-marketing-analytics-metrics-goals-kpis/?ysclid=lbbylgx6fw730733283
5. https://emailmatrix.ru/blog/the-value-of-calculating-revenue-per-email/?ysclid=lbbykxcwpe466291172

# Comparison test and control groups

In [45]:
results_control = sqlContext.sql("""WITH single_cards as (
                                SELECT crds.cust_id , control_group_flg , min(crds.open_dt) as open_dt
                                FROM crds 
                                INNER JOIN (SELECT cust_id , control_group_flg FROM comms GROUP BY cust_id , control_group_flg) as cm 
                                ON crds.cust_id = cm.cust_id AND cm.control_group_flg = 1  
                                GROUP BY crds.cust_id, cm.control_group_flg
                            )
                            SELECT a.cust_id 
                                , a.open_dt 
                                , sum(coalesce(all_t.txn_sum,0)) as all_txn_sum
                                , a.campaign_txn_sum
                                , a.all_trans_count as campaign_trans_count
                                , a.flag_success_campaign
                                , a.control_group_flg
                            FROM (
                                SELECT cust_id 
                                    , open_dt 
                                    , sum(CASE WHEN main.all_trans_count >= 3 
                                                THEN txn_sum
                                                ELSE 0
                                          END) AS campaign_txn_sum
                                    , all_trans_count
                                    , CASE WHEN main.all_trans_count >= 3 
                                            THEN 1
                                            ELSE 0
                                      END AS flag_success_campaign
                                    , control_group_flg
                                FROM (
                                    SELECT crds.cust_id 
                                        , crds.open_dt 
                                        , trans.transaction_dt 
                                        , trans.mcc_id 
                                        , trans.txn_sum
                                        , count(trans.cust_id) OVER (PARTITION BY crds.cust_id) as all_trans_count
                                        , crds.control_group_flg
                                    FROM single_cards as crds
                                    LEFT JOIN trans 
                                    ON crds.cust_id = trans.cust_id  
                                    AND trans.txn_sum >= 500 
                                    AND crds.open_dt <= trans.transaction_dt
                                    AND trans.transaction_dt < date_add(crds.open_dt, 14) 
                                ) main
                                GROUP BY cust_id 
                                    , open_dt 
                                    , all_trans_count
                                    , CASE WHEN main.all_trans_count >= 3 
                                            THEN 1
                                            ELSE 0
                                      END
                                    , control_group_flg
                            ) a
                            LEFT JOIN trans as all_t
                                ON a.cust_id = all_t.cust_id 
                            group by a.cust_id 
                                , a.open_dt 
                                , a.campaign_txn_sum
                                , a.all_trans_count
                                , a.flag_success_campaign
                                , a.control_group_flg
                            """)
results_control.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[cust_id#119, open_dt#935, campaign_txn_sum#931, all_trans_count#930L, flag_success_campaign#932, control_group_flg#938L], functions=[sum(coalesce(txn_sum#957, 0.0))])
   +- HashAggregate(keys=[cust_id#119, open_dt#935, knownfloatingpointnormalized(normalizenanandzero(campaign_txn_sum#931)) AS campaign_txn_sum#931, all_trans_count#930L, flag_success_campaign#932, control_group_flg#938L], functions=[partial_sum(coalesce(txn_sum#957, 0.0))])
      +- Project [cust_id#119, open_dt#935, campaign_txn_sum#931, all_trans_count#930L, flag_success_campaign#932, control_group_flg#938L, txn_sum#957]
         +- SortMergeJoin [cust_id#119], [cust_id#954], LeftOuter
            :- Sort [cust_id#119 ASC NULLS FIRST], false, 0
            :  +- HashAggregate(keys=[cust_id#119, open_dt#935, all_trans_count#930L, _groupingexpression#978, control_group_flg#938L], functions=[sum(CASE WHEN (all_trans_count#930L >= 3) THEN txn_su

We need to compare control and test groups: we will do it with the conversion rate and the comparison of the financial outcome (was there a financial uplift because of the compaign).
In order to calculate the conversion rates, we will use the following formula:
target action/number of people



In [61]:
#control group 

target_action_control = results_control.filter(results_control.flag_success_campaign == 1).select(results_control.flag_success_campaign).count()
numberofpeople_control =results_control.select(results_control.flag_success_campaign).count()
ConvR_control= target_action_control/numberofpeople_control 


In [75]:
#test group 

target_action_test = results.filter(results.flag_success_campaign == 1).select(results.flag_success_campaign).count()
numberofpeople_test =results.select(results.flag_success_campaign).count()
ConvR_test= target_action_test/numberofpeople_test


In [106]:
numberofpeople_test

20219

In [107]:
numberofpeople_control

661

In [80]:
print("the conversion rate of control group is", ConvR_control)
print("the conversion rate of test group is", ConvR_test)

the conversion rate of control group is 0.32526475037821484
the conversion rate of test group is 0.2655423116870271


In [100]:
revenue_test =(results.select(results.campaign_txn_sum).agg(sum("campaign_txn_sum")).show())

+---------------------+
|sum(campaign_txn_sum)|
+---------------------+
|  9.664379336999992E7|
+---------------------+



In [102]:
revenue_control =(results_control.select(results_control.campaign_txn_sum).agg(sum("campaign_txn_sum")).show())

+---------------------+
|sum(campaign_txn_sum)|
+---------------------+
|    3316432.380000001|
+---------------------+



In [103]:
#bank's income is 1% of transactions
profit_test= np.round(9.664379336999992E7)*0.01
profit_control=np.round(3316432.380000001)*0.01

In [110]:
uplift =(ConvR_test - ConvR_control)/ConvR_test

In [111]:
print("uplift is",uplift)

uplift is -0.22490742929728538


In [ ]:
z_score = (rate_b - rate_a) / np.sqrt(std_a**2 + std_b**2) 
print(f"z-score is {z_score:0.3f}, with p-value {norm().sf(z_score):0.3f}") 
 
# But I want a plot as well 
p = norm(rate_b - rate_a, np.sqrt(std_a**2 + std_b**2))

The final results show that the conversion metrics are better in the control group, the uplift is negative

Lets check the results:

In [122]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
n_con = 661
n_treat = 20219
successes = [215, 5369]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 3.41
p-value: 0.001
ci 95% for control group: [0.290, 0.361]
ci 95% for treatment group: [0.259, 0.272]


With the confidence level 95% we can state that the results are statistically significant 

# Creation of synthetic control group

In [ ]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
import numpy as np
from pyspark.sql import functions as F, types as T
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT 

from sklearn.neighbors import NearestNeighbors

df_ftrs = df_ftrs.withColumn("turnover",col("turnover").cast('double'))

In [ ]:
group_cols = ["cust_id", "involve_cd", "client_segment", "gender", "age_gr"]

df_ftrs_pivot = df_ftrs.groupBy(group_cols).pivot("mnth_gen").sum("turnover")
df_ftrs_pivot.printSchema()
del df_ftrs

In [ ]:
df_ftrs_pivot = df_ftrs_pivot.drop("0") 

In [ ]:
df_ftrs_pivot = df_ftrs_pivot.withColumn("involve_cd",col("involve_cd").cast('int'))
df_ftrs_pivot.printSchema()
df_ftrs_pivot = df_ftrs_pivot.na.fill(value=0)

In [ ]:
indexer = StringIndexer(inputCols=["client_segment", "gender", "age_gr"], outputCols=["client_segment_index", "gender_index", "age_gr_index"], handleInvalid='skip', stringOrderType='frequencyDesc') 
df_ftrs_pivot_indexed = indexer.fit(df_ftrs_pivot).transform(df_ftrs_pivot) 
df_ftrs_pivot_indexed.printSchema()
del df_ftrs_pivot


In [ ]:
df_ftrs_pivot_indexed = df_ftrs_pivot_indexed.drop("client_segment", "gender", "age_gr")
df_ftrs_pivot_indexed.printSchema()

In [ ]:
vecAssembler = VectorAssembler(inputCols=["client_segment_index", "gender_index", "age_gr_index", "2021-01-01", "2021-02-01", "2021-03-01", "2021-04-01", "2021-05-01", "2021-06-01", "2021-07-01", "2021-08-01", "2021-09-01", "2021-10-01"], outputCol="features", handleInvalid="skip")

In [ ]:
to_dense_vector_udf = F.udf(lambda l: Vectors.dense(l), VectorUDT())

df_ftrs_pivot_indexed_vectorized = vecAssembler.transform(df_ftrs_pivot_indexed)
df_ftrs_pivot_indexed_vectorized = df_ftrs_pivot_indexed_vectorized.withColumn('features', F.array(*df_ftrs_pivot_indexed_vectorized.columns))
df_ftrs_pivot_indexed_vectorized.printSchema()

In [ ]:
test_ftrs_pivot_indexed_vectorized = np.array(df_ftrs_pivot_indexed_vectorized.select("features").filter(df_ftrs_pivot_indexed_vectorized.involve_cd == 1).limit(100).collect())

In [ ]:
control_ftrs_pivot_indexed_vectorized = np.array(df_ftrs_pivot_indexed_vectorized.select("features").filter(df_ftrs_pivot_indexed_vectorized.involve_cd == 0).limit(100).collect()).reshape(1, -1)

In [ ]:
def get_matching_pairs(treated_x, non_treated_x):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(non_treated_x)
    distances, indices = nbrs.kneighbors(treated_x)
    return indices

get_matching_pairs(test_ftrs_pivot_indexed_vectorized, control_ftrs_pivot_indexed_vectorized)

Here we generated matching pairs using NearestNeighbors algorithm


# Promo mechanics

The openability can also be increased with the help of a header — this is the text that mail programs show after the subject of the letter. A preheader can also be useful and informative. For example, it can be interesting to continue and reveal the subject of the letter.

Indicators can be improved by segmenting subscribers. A large database is divided into several, and their own letters are prepared for each group. This way the messages become more relevant for each segment

We need to choose the optimal frequency of sending and not send emails too often or too rarely.In the first case, subscribers will forget what this newsletter is about, in the second case, they may consider it spam

Also some another things can be considered: email design, easy-to-click call to action